In [ ]:
### PROJEKT ANALIZA DANYCH W R I PYTHON

#INSTALACJA PAKIETOW
#!pip install pandas
#!pip install matplotlib
%pip install seaborn
%pip install openpyxl
%pip install scikit-learn

# ZALADOWANIE PAKIETOW
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from math import pi
import numpy as np

# ZALADOWANIE DANYCH
# Upewnij się, że plik 'NBA - stats.xlsx' jest w tym samym folderze!
df = pd.read_excel('NBA - stats.xlsx')


In [ ]:
#1 STRUKTURA DANYCH
#1.1 PIERWSZE 5 WIERSZY
print("\n### 1.1 Pierwsze 5 wierszy danych:")
print(df.head())

#1.2 SPRAWDZENIE BRAKUJACYCH WARTOSCI
print("\n### 1.2 Liczba brakujących wartości w każdej kolumnie:")
print(df.isnull().sum())

#1.3 Zaiana nazwy kolumny
df.rename(columns={'Unnamed: 30': 'Position'}, inplace=True)

In [ ]:
#2 STATYSTYKA OPISOWA
print("\n### 2.1 Podstawowe statystyki opisowe (kolumny numeryczne):")
print(df.describe())

#2.1 USTAWIENIE NUMERYCZNYCH KOLUMN
for col in ['Salary', 'Age', 'PTS']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df_clean = df.dropna(subset=['Salary', 'Age', 'PTS'])

# 2.2 NAJSTARSZY ORAZ NAJMLODSZY GRACZ
oldest_player = df_clean.loc[df_clean['Age'].idxmax(), ['Player', 'Age']]
youngest_player = df_clean.loc[df_clean['Age'].idxmin(), ['Player', 'Age']]

# 2.3 NAJWIEKSZA I NAJMNIEJSZA PENSJA
max_salary_player = df_clean.loc[df_clean['Salary'].idxmax(), ['Player', 'Salary']]
min_salary_player = df_clean.loc[df_clean['Salary'].idxmin(), ['Player', 'Salary']]

# 2.4 GRACZ Z NAJWIEKSZA LICZBA PUNKTOW
max_pts_player = df_clean.loc[df_clean['PTS'].idxmax(), ['Player', 'PTS']]

# 2.5 Średnia dla wynagrodzenia, wieku, liczby punktow
avg_salary = df_clean['Salary'].mean()
avg_age = df_clean['Age'].mean()
avg_pts = df_clean['PTS'].mean()

# 2.6 SREDNIA WYNAGRODZENIA W ZALEZNOSCI OD ZESPOLU
team_avg_salary = df_clean.groupby('Team')['Salary'].mean().sort_values(ascending=False)

# 2.7 WIEK NAJWIEKSZEGO SREDNIEGO WYNAGRODZENIA
age_avg_salary = df_clean.groupby('Age')['Salary'].mean()
max_avg_salary_age = age_avg_salary.idxmax()
max_avg_salary_value = age_avg_salary.max()

# 2.8 WYSWIETLANIE
print("\n--- Statystyki Opisowe Graczy NBA ---")

print("\n1. Najstarszy i Najmłodszy Gracz:")
print(f"Najstarszy Gracz: {oldest_player['Player']} ({oldest_player['Age']:.0f} lat)")
print(f"Najmłodszy Gracz: {youngest_player['Player']} ({youngest_player['Age']:.0f} lat)")

print("\n2. Najwyższa i Najniższa Pensja:")
print(f"Najwyższa Pensja: {max_salary_player['Player']} (${max_salary_player['Salary']:,.0f})")
print(f"Najniższa Pensja: {min_salary_player['Player']} (${min_salary_player['Salary']:,.0f})")

print("\n3. Gracz z Największą Liczbą Punktów (PTS):")
print(f"Gracz: {max_pts_player['Player']} ({max_pts_player['PTS']:.1f} PTS)")

print("\n4. Średnie Wartości:")
print(f"Średnie Wynagrodzenie: ${avg_salary:,.0f}")
print(f"Średni Wiek: {avg_age:.2f} lat")
print(f"Średnia Liczba Punktów (PTS): {avg_pts:.2f}")

print("\n5. Średnie Wynagrodzenie w Zależności od Zespołu (Top 5):")
# POPRAWKA: Zastąpienie .to_markdown() przez .to_string(header=False)
print(team_avg_salary.head(5).apply(lambda x: f"${x:,.0f}").to_string(header=False))

print("\n6. Wiek ze Średnio Największym Wynagrodzeniem:")
print(f"Wiek: {max_avg_salary_age:.0f} lat (Średnia Pensja: ${max_avg_salary_value:,.0f})")

In [ ]:
#3 WIZUALIZACJE 

# --- 2. Wykres 1: Średnia Pensja w Zależności od Zespołu (Bar Chart) ---
team_avg_salary = df_clean.groupby('Team')['Salary'].mean().sort_values(ascending=False)

plt.figure(figsize=(14, 7))
sns.barplot(x=team_avg_salary.index, y=team_avg_salary.values, palette="viridis")
plt.xticks(rotation=75, fontsize=8)
plt.title('Wykres 1: Średnie Wynagrodzenie (USD) w Zależności od Zespołu (Posortowane)', fontsize=14)
plt.xlabel('Zespół NBA')
plt.ylabel('Średnie Wynagrodzenie (USD)')

# Formatowanie osi Y na miliony dolarów
formatter = plt.FuncFormatter(lambda x, pos: f'${x/1e6:,.0f}M')
plt.gca().yaxis.set_major_formatter(formatter)

plt.tight_layout()
plt.savefig('wykres_1_pensja_zespol.png')
plt.close()

# --- 3. Wykres 2: Punkty (PTS) w Zależności od Wieku (Scatter Plot z linią trendu) ---
plt.figure(figsize=(10, 6))
# regplot z seaborn automatycznie dodaje linię regresji
sns.regplot(x='Age', y='PTS', data=df_clean, scatter_kws={'alpha':0.6}, line_kws={"color": "red"})
plt.title('Wykres 2: Punkty (PTS) w Zależności od Wieku (z Linią Trendu)', fontsize=14)
plt.xlabel('Wiek (lata)')
plt.ylabel('Punkty na Mecz (PTS)')
plt.tight_layout()
plt.savefig('wykres_2_punkty_wiek.png')
plt.close()

# --- 4. Wykres 3: Pensja (Salary) w Zależności od Punktów (PTS) z Wpływem Wieku (Scatter Plot) ---
plt.figure(figsize=(10, 6))
# Użycie wieku do koloru (hue) i rozmiaru (size) punktu dla dodatkowej informacji
sns.scatterplot(x='PTS', y='Salary', data=df_clean, alpha=0.6, hue='Age', size='Age', sizes=(20, 200), palette='coolwarm')
plt.title('Wykres 3: Pensja (Salary) vs Punkty (PTS) z Wpływem Wieku', fontsize=14)
plt.xlabel('Punkty na Mecz (PTS)')
plt.ylabel('Wynagrodzenie (USD)')

# Formatowanie osi Y na miliony dolarów
formatter = plt.FuncFormatter(lambda x, pos: f'${x/1e6:,.0f}M')
plt.gca().yaxis.set_major_formatter(formatter)

plt.legend(title='Wiek', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('wykres_3_pensja_punkty.png')
plt.close()

# --- 5. Wykres 4 (Ciekawy): Średnie Wynagrodzenie w Zależności od Wieku (Line Plot) ---
age_avg_salary = df_clean.groupby('Age')['Salary'].mean()

plt.figure(figsize=(10, 6))
sns.lineplot(x=age_avg_salary.index, y=age_avg_salary.values, marker='o', color='darkred')
plt.title('Wykres 4: Średnie Wynagrodzenie (USD) a Wiek Gracza', fontsize=14)
plt.xlabel('Wiek (lata)')
plt.ylabel('Średnie Wynagrodzenie (USD)')
# Ustawienie etykiet X co 2 lata dla czytelności
plt.xticks(np.arange(df_clean['Age'].min(), df_clean['Age'].max()+1, 2))

# Formatowanie osi Y na miliony dolarów
formatter = plt.FuncFormatter(lambda x, pos: f'${x/1e6:,.0f}M')
plt.gca().yaxis.set_major_formatter(formatter)

plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('wykres_4_srednia_pensja_wiek.png')
plt.close()

# Wybór graczy do porównania
players_to_compare = ['Stephen Curry', 'Nikola Jokic', 'Giannis Antetokounmpo']
stats_cols = ['PTS', 'AST', 'REB', 'STL', 'BLK', '3PM']
categories = stats_cols

# Normalizacja Danych (Min-Max Scaling)
# Znormalizujemy dane na skali od 0 do 1 względem całego zbioru danych.
df_normalized = df_clean.copy()
for col in stats_cols:
    min_val = df_normalized[col].min()
    max_val = df_normalized[col].max()
    # Wzór normalizacji: (wartość - min) / (max - min)
    df_normalized[col] = (df_normalized[col] - min_val) / (max_val - min_val)

# Dane dla wykresu radarowego (tylko wybrani gracze z znormalizowanymi statystykami)
radar_data = df_normalized[df_normalized['Player'].isin(players_to_compare)][['Player'] + stats_cols].set_index('Player')

if radar_data.empty:
    print(f"BŁĄD: Nie znaleziono graczy ({players_to_compare}) do porównania. Sprawdź, czy ich nazwy są poprawne w pliku.")
    exit()

# --- 3. Generowanie Wykresu Radarowego ---

def create_radar_chart(df_data, categories, title):
    """Tworzy i zapisuje wykres radarowy."""
    
    N = len(categories)
    
    # Tworzenie kątów dla każdej osi
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1] # Zamknięcie koła
    
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
    
    # Ustawienie orientacji osi
    ax.set_theta_offset(pi / 2) # Pierwsza oś na górze
    ax.set_theta_direction(-1) # Kierunek rysowania zgodny z ruchem wskazówek zegara
    
    # Etykiety na obwodzie
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, fontsize=12, weight='bold')
    
    # Etykiety dla osi promieniowej (wartości od 0 do 1)
    ax.set_yticks(np.arange(0, 1.1, 0.2))
    ax.set_yticklabels([f"{val:.1f}" for val in np.arange(0, 1.1, 0.2)], color="grey", size=10)
    ax.set_ylim(0, 1)
    
    # Rysowanie poligonów
    for i, player in enumerate(df_data.index):
        values = df_data.loc[player].values.flatten().tolist()
        values += values[:1] # Zamknięcie poligonu
        
        # Kolory dla wybranych graczy (opcjonalne, ale dodaje czytelności)
        color_map = {'Stephen Curry': 'gold', 'Nikola Jokic': 'blue', 'Giannis Antetokounmpo': 'green'}
        color = color_map.get(player, f'C{i}')

        ax.plot(angles, values, linewidth=2, linestyle='solid', label=player, color=color)
        ax.fill(angles, values, color=color, alpha=0.25)
        
    ax.set_title(title, y=1.1, fontsize=16)
    ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1), fontsize=10)
    
    plt.savefig('wykres_radarowy_gwiazdy_nba.png')
    plt.close()

# Wywołanie funkcji
create_radar_chart(radar_data, categories, 'Wykres Radarowy: Porównanie Statystyk Gwiazd NBA (Znormalizowane)')
print("Wygenerowano wykres radarowy: wykres_radarowy_gwiazdy_nba.png.")

#DO POPRWAY WIZUALIZACJE + DODANIE DO STATYSTYKI OPISOWEJ